In [2]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
import seaborn as sns
import pandas as pd
from dhdrnet.Dataset import LUTDataset
from dhdrnet.util import ROOT_DIR, DATA_DIR
import matplotlib.pyplot as plt

In [5]:
precomputed = ROOT_DIR / "precomputed_data"
ds = LUTDataset(
        choice_path=(precomputed / "store_finer_2020-07-04.csv"),
        exposure_path= DATA_DIR / "correct_exposures" / "exposures",
        name_list=ROOT_DIR/"train.txt"
    )

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [16]:
df = (
        pd.read_csv(precomputed / "store_finer_2020-07-04.csv", usecols=["name", "metric", "score", "ev1", "ev2"])
        .set_index("name")
        .sort_index()
     )
df

,metric,ev1,ev2,score
name,,,,
0006_20160721_163256_525,mse,-3.0,-2.75,9865.486670
0006_20160721_163256_525,ssim,0.0,2.50,0.969474
0006_20160721_163256_525,mse,0.0,2.50,546.063286
0006_20160721_163256_525,perceptual,0.0,2.25,0.066602
0006_20160721_163256_525,ssim,0.0,2.25,0.968456
...,...,...,...,...
synthetic_ramp_night_motion_blur,perceptual,0.0,2.25,0.091324
synthetic_ramp_night_motion_blur,mse,0.0,2.50,732.314913
synthetic_ramp_night_motion_blur,ssim,0.0,2.50,0.862310


In [17]:
bdf = df[(df["ev1"] == 0) | (df["ev2"] == 0)].copy()
bdf["ev"] = bdf[["ev1","ev2"]].apply(lambda evs: [ev for ev in evs if ev != 0][0],axis=1)
bdf = bdf.drop(columns=["ev1", "ev2"])
mse_bdf = bdf[bdf["metric"] == "mse"]
bdf

,metric,score,ev
name,,,
0006_20160721_163256_525,ssim,0.969474,2.50
0006_20160721_163256_525,mse,546.063286,2.50
0006_20160721_163256_525,perceptual,0.066602,2.25
0006_20160721_163256_525,ssim,0.968456,2.25
0006_20160721_163256_525,mse,662.772249,2.25
...,...,...,...
synthetic_ramp_night_motion_blur,ssim,0.886127,2.25
synthetic_ramp_night_motion_blur,perceptual,0.091324,2.25
synthetic_ramp_night_motion_blur,mse,732.314913,2.50


In [18]:
by_ev = bdf.pivot_table(index="name", columns=["metric", "ev"], values="score")
by_ev["mse"].idxmin(axis=1)


name
0006_20160721_163256_525                   4.00
0006_20160721_170707_736                   3.25
0006_20160721_172953_515                   4.25
0006_20160721_175239_909                   3.75
0006_20160721_175907_041                   4.25
                                           ... 
synthetic_points_hawk_day_motion_blur      3.75
synthetic_points_hawk_night_motion_blur    4.25
synthetic_ramp_day                        -1.00
synthetic_ramp_day_motion_blur            -1.00
synthetic_ramp_night_motion_blur           1.75
Length: 2184, dtype: float64

In [19]:
mse_bdf.pivot(columns="ev", values="score").idxmin(axis=1)

name
0006_20160721_163256_525                   4.00
0006_20160721_170707_736                   3.25
0006_20160721_172953_515                   4.25
0006_20160721_175239_909                   3.75
0006_20160721_175907_041                   4.25
                                           ... 
synthetic_points_hawk_day_motion_blur      3.75
synthetic_points_hawk_night_motion_blur    4.25
synthetic_ramp_day                        -1.00
synthetic_ramp_day_motion_blur            -1.00
synthetic_ramp_night_motion_blur           1.75
Length: 2184, dtype: float64

In [20]:
by_ev.loc[:, "mse"]

ev,-3.00,-2.75,-2.50,-2.25,-2.00,-1.75,-1.50,-1.25,-1.00,-0.75,...,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25
name,,,,,,,,,,,,,,,,,,,,,
0006_20160721_163256_525,669.071654,699.836457,723.294404,731.829233,722.200191,634.889757,634.220030,588.496416,510.766765,425.198371,...,752.889053,662.772249,546.063286,431.307934,342.504634,283.440348,248.290604,233.099941,228.593360,229.639263
0006_20160721_170707_736,1564.444742,1620.747432,1666.827206,1694.827552,1703.434451,1598.156467,1594.859254,1543.512098,1416.238552,1255.256251,...,372.399781,335.060676,288.831202,239.248197,191.736304,162.033350,167.842763,204.894840,266.079117,342.824356
0006_20160721_172953_515,2707.720105,2739.095790,2765.210355,2760.221485,2733.397875,2595.181069,2591.458434,2468.884454,2310.264377,2118.547153,...,425.801535,402.205455,423.542583,481.057105,542.468597,579.089100,533.290667,417.449806,323.527967,308.786930
0006_20160721_175239_909,4833.678364,4931.049788,5023.474798,5096.684604,5149.722950,5132.641241,5127.621899,5039.689928,4886.487238,4668.281993,...,565.067743,389.886246,257.681149,170.742819,123.824036,105.273205,99.773111,97.861535,113.651506,189.986430
0006_20160721_175907_041,3445.913765,3484.464974,3508.670120,3506.884531,3475.759575,3305.814958,3295.291553,3189.912140,3018.660781,2812.354843,...,604.458380,493.163809,415.175241,361.472923,319.402364,288.606714,275.766842,274.817158,273.081426,268.427820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
synthetic_points_hawk_day_motion_blur,9549.787285,9616.957304,9702.412601,9789.976959,9838.795522,9870.674157,9778.043692,9775.709732,9681.766884,9493.410490,...,3238.620243,2570.854984,1928.538583,1348.103066,847.423600,447.837387,173.198507,53.542805,106.342976,312.289576
synthetic_points_hawk_night_motion_blur,9100.884809,9150.857141,9204.525415,9256.408295,9336.651105,9344.758113,9365.465235,9357.539050,9318.120591,9233.594149,...,4117.931006,3446.512400,2803.061730,2203.537571,1655.339579,1173.081054,762.147347,434.131829,203.471327,82.635917
synthetic_ramp_day,582.577840,585.154776,580.085023,557.480435,528.451155,488.213527,449.746971,419.779580,408.363118,433.778416,...,1380.949946,1363.672676,1348.305105,1333.135083,1315.994506,1297.478384,1278.704576,1259.128457,1242.134971,1226.111837


In [21]:
by_ev["mse"]

ev,-3.00,-2.75,-2.50,-2.25,-2.00,-1.75,-1.50,-1.25,-1.00,-0.75,...,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25
name,,,,,,,,,,,,,,,,,,,,,
0006_20160721_163256_525,669.071654,699.836457,723.294404,731.829233,722.200191,634.889757,634.220030,588.496416,510.766765,425.198371,...,752.889053,662.772249,546.063286,431.307934,342.504634,283.440348,248.290604,233.099941,228.593360,229.639263
0006_20160721_170707_736,1564.444742,1620.747432,1666.827206,1694.827552,1703.434451,1598.156467,1594.859254,1543.512098,1416.238552,1255.256251,...,372.399781,335.060676,288.831202,239.248197,191.736304,162.033350,167.842763,204.894840,266.079117,342.824356
0006_20160721_172953_515,2707.720105,2739.095790,2765.210355,2760.221485,2733.397875,2595.181069,2591.458434,2468.884454,2310.264377,2118.547153,...,425.801535,402.205455,423.542583,481.057105,542.468597,579.089100,533.290667,417.449806,323.527967,308.786930
0006_20160721_175239_909,4833.678364,4931.049788,5023.474798,5096.684604,5149.722950,5132.641241,5127.621899,5039.689928,4886.487238,4668.281993,...,565.067743,389.886246,257.681149,170.742819,123.824036,105.273205,99.773111,97.861535,113.651506,189.986430
0006_20160721_175907_041,3445.913765,3484.464974,3508.670120,3506.884531,3475.759575,3305.814958,3295.291553,3189.912140,3018.660781,2812.354843,...,604.458380,493.163809,415.175241,361.472923,319.402364,288.606714,275.766842,274.817158,273.081426,268.427820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
synthetic_points_hawk_day_motion_blur,9549.787285,9616.957304,9702.412601,9789.976959,9838.795522,9870.674157,9778.043692,9775.709732,9681.766884,9493.410490,...,3238.620243,2570.854984,1928.538583,1348.103066,847.423600,447.837387,173.198507,53.542805,106.342976,312.289576
synthetic_points_hawk_night_motion_blur,9100.884809,9150.857141,9204.525415,9256.408295,9336.651105,9344.758113,9365.465235,9357.539050,9318.120591,9233.594149,...,4117.931006,3446.512400,2803.061730,2203.537571,1655.339579,1173.081054,762.147347,434.131829,203.471327,82.635917
synthetic_ramp_day,582.577840,585.154776,580.085023,557.480435,528.451155,488.213527,449.746971,419.779580,408.363118,433.778416,...,1380.949946,1363.672676,1348.305105,1333.135083,1315.994506,1297.478384,1278.704576,1259.128457,1242.134971,1226.111837


FileNotFoundError: [Errno 2] No such file or directory: '/data/Datasets/HDR+/correct_exposure/exposures/0006_20160721_163256_525[0.0].png'

In [10]:
from more_itertools import flatten
train_names = list(flatten(pd.read_csv(ROOT_DIR/"train.txt", header=None).to_numpy()))
test_names = list(flatten(pd.read_csv(ROOT_DIR/"test.txt", header=None).to_numpy()))

In [11]:
train = by_ev.loc[train_names]

In [12]:
for i, n in enumerate(train["mse"].index.array):
    if n == "c1b1_20150312_205541_959":
        print(i)
        break

1464


In [17]:
ds[0]

(tensor([[[0.1765, 0.1765, 0.1765,  ..., 0.1725, 0.1686, 0.1569],
          [0.1804, 0.1804, 0.1843,  ..., 0.1647, 0.1569, 0.1412],
          [0.2078, 0.2039, 0.2039,  ..., 0.1490, 0.1490, 0.1490],
          ...,
          [0.0902, 0.0745, 0.0824,  ..., 0.1922, 0.1804, 0.1804],
          [0.0902, 0.0824, 0.0824,  ..., 0.1922, 0.1843, 0.1765],
          [0.0902, 0.0824, 0.0706,  ..., 0.1882, 0.1804, 0.1765]],
 
         [[0.1882, 0.1961, 0.2000,  ..., 0.1725, 0.1647, 0.1608],
          [0.2039, 0.2078, 0.2039,  ..., 0.1686, 0.1569, 0.1490],
          [0.2275, 0.2353, 0.2353,  ..., 0.1529, 0.1569, 0.1686],
          ...,
          [0.1059, 0.0902, 0.0863,  ..., 0.1490, 0.1451, 0.1451],
          [0.1020, 0.0980, 0.0824,  ..., 0.1451, 0.1451, 0.1412],
          [0.0980, 0.1020, 0.0784,  ..., 0.1451, 0.1373, 0.1373]],
 
         [[0.2078, 0.1961, 0.2000,  ..., 0.1608, 0.1451, 0.1412],
          [0.2118, 0.2118, 0.2078,  ..., 0.1569, 0.1569, 0.1490],
          [0.2431, 0.2431, 0.2392,  ...,

In [48]:
list(flatten(train_names.to_numpy()))

['0039_20141008_195729_671',
 '0094_20150412_125907_902',
 'bee7_20150818_113445_197',
 'c483_20150901_221141_886',
 'c1b1_20150311_195432_970',
 '0155_20160927_145933_894',
 'c1b1_20150312_141022_171',
 '0039_20141008_202923_344',
 '5066_20160416_114622_891',
 '0155_20160819_145132_296',
 '5030_20150913_114646_870',
 '0919_20150906_131434_544',
 '5066_20160722_141703_425',
 '9bf4_20150818_173321_675',
 '6G7M_20150321_124154_288',
 '6G7M_20150325_084014_332',
 '4KK2_20150822_213656_916',
 '0127_20161018_121045_621',
 '0155_20161002_142748_614',
 '0030_20151217_221631_804',
 '0919_20150906_131846_532',
 '0127_20161022_144117_906',
 '4742_20150918_185536_804',
 '4742_20150919_084156_722',
 '33TJ_20150616_174447_472',
 '33TJ_20150821_115332_658',
 '0543_20150918_182219_866',
 '4KK2_20150829_142249_654',
 '4KK2_20150822_224325_221',
 '0039_20141009_162226_419',
 '1125_20151227_114839_809',
 '5a9e_20141009_133737_056',
 '0272_20151216_000440_169',
 '0543_20150917_124715_973',
 '5030_2015091